In [10]:
#IMPORTS

import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

import requests
from bs4 import BeautifulSoup
import time

In [11]:
df = pd.DataFrame(columns=['href_list']) # creates master dataframe 

# Chromedriver Path
PATH = "/Users/javm/Desktop/chromedriver_mac_arm64/chromedriver"

# this creates/opens Chrome browser
driver = webdriver.Chrome(PATH)

#creating list of href
href_list = []

for listings in range(21,36):
    page_num = str(listings)
    url = 'https://www.barnesandnoble.com/b/books/_/N-1z13w9sZ1sZ29Z8q8?Nrpp=20&page=' + page_num
    driver.get(url)
    # freeze/halt python so url loads completely
    driver.implicitly_wait(60)
    
    #find hrefs
    div_elements = driver.find_elements(By.CSS_SELECTOR, "div.product-shelf-title.product-info-title.pt-xs")
    for div_element in div_elements:
        elements = div_element.find_elements(By.TAG_NAME, "a")
        for element in elements:
            href = element.get_attribute('href')
            href_list.append(href)

    urls_df = pd.DataFrame(href_list, columns=['href_list']) # creates master dataframe
    df = pd.concat([df, urls_df], ignore_index=True)
    
driver.close()


/var/folders/6g/zn0bw_6s1ws2qwbb44k4ppkw0000gn/T/ipykernel_20429/2390757508.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [12]:
len(href_list)

300

In [13]:
# Chromedriver Path
PATH = "/Users/javm/Desktop/chromedriver_mac_arm64/chromedriver"

# this creates/opens Chrome browser
driver = webdriver.Chrome(PATH)

book_details = []

for index, href in enumerate(href_list):
    driver.get(href)
    driver.implicitly_wait(60)

    # find titles
    titles = driver.find_elements(By.TAG_NAME, 'h1')
    title_list = []
    for t in range(len(titles)):
        title_list.append(titles[t].text)

    if not title_list:
        continue

    # find author
    author_list = []
    try:
        author_elems = driver.find_elements(By.CSS_SELECTOR, '.contributors.pdp-book-author a')
        for author_elem in author_elems:
            author_list.append(author_elem.text)
    except NoSuchElementException:
        author_list = [np.nan]

    # find publisher
    try:
        publisher_span = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="publisher"]')
        publisher = publisher_span.text
    except NoSuchElementException:
        publisher = np.nan

    # find blurb
    try:
        parent_div = driver.find_element(By.XPATH, '//*[@id="overviewSection"]')
        blurb_text = parent_div.text
    except NoSuchElementException:
        blurb_text = np.nan
    
    book = [title_list[1] if len(title_list) > 1 else title_list[0], author_list, publisher, blurb_text]
    book_details.append(book)

driver.quit()

BOOK_DF_2 = pd.DataFrame(book_details, columns=['title', 'authors', 'publisher', 'blurb'])



/var/folders/6g/zn0bw_6s1ws2qwbb44k4ppkw0000gn/T/ipykernel_20429/2600020984.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [14]:
BOOK_DF_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      300 non-null    object
 1   authors    300 non-null    object
 2   publisher  300 non-null    object
 3   blurb      299 non-null    object
dtypes: object(4)
memory usage: 9.5+ KB


In [15]:
BOOK_DF_2.to_csv('books_2.csv', index = False)

In [16]:
BOOK_DF_2

,title,authors,publisher,blurb
0,Familias de la A a la Z / Families from A to Z,"[, Raquel Díaz Reguera]",PRH Grupo Editorial,Overview\nCada familia es un mundo… y el mundo...
1,El ladrón de rostros (Inspectora Ane Cestero 3),"[, Ibon Martín]",Penguin Random House Grupo Editorial España,Overview\nRegresa Ibon Martín. Regresa el mejo...
2,Un Día Con Grace,"[, Samantha Melendez]",Barnes & Noble Press,Overview\nUna historia divertida sigue a una n...
3,Aventuras en la bañera de Andrés el bebé y Mic...,"[, , , Tom Haydon, Annika Haydon, Magdalena Za...",TomAnni Publishing,"Overview\nEstos son Andrés y Mickey, el duo má..."
4,Reina Valera Revisada Biblia de Referencia Tho...,"[, , , Charles Thompson, Reina Valera Revisada...",Vida,Overview\nUna herramienta fácil de utilizar pa...
...,...,...,...,...
295,El cisne de chatarra: la limpieza de un río,"[, , Joanne Linden, Estrellita Caracol (Illust...",Barefoot Books,Overview\nWhen community members work together...
296,"Reina Valera Revisada, Biblia de Referencia Th...","[, , , Charles Thompson, Reina Valera Revisada...",Vida,Overview\nUna herramienta fácil de utilizar pa...
297,Tragicomedia Mexicana 1: La vida en México de ...,"[, José Agustín]",PRH Grupo Editorial,Overview\nLa radiografía de un país hecha por ...
298,Frida McMoon y los aprendices del tiempo / Fri...,"[, , Pedro Mañas, Laia Ferraté (Illustrator)]",PRH Grupo Editorial,Overview\nUna aventura de la heroína más espec...
